In [65]:
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
import os
import pandas as pd
from tqdm import tqdm
from skimage import io
from skimage.transform import rescale, resize, downscale_local_mean
from torch.utils.data import (
    Dataset,
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

In [66]:
filenames = os.listdir("E:\\JK\\Vt\\dataset_vit_orginal - Copy\\train")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'cluster':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'image': filenames,
    'labels': categories
    
})
df


,image,labels
0,cluster.1 (2).jpg,1
1,cluster.1.jpg,1
2,cluster.10 (2).jpg,1
3,cluster.10.jpg,1
4,cluster.104 (2).jpg,1
...,...,...
1789,non_cluster.97.jpg,0
1790,non_cluster.98 (2).jpg,0
1791,non_cluster.98.jpg,0
1792,non_cluster.99 (2).jpg,0


In [67]:
df['image']=df['image'].astype(str)

In [68]:
df.to_csv('file2.csv', index=False)

In [76]:
files = os.listdir("E:\\JK\\Vt\\dataset_vit_orginal - Copy\\train")
files

['cluster.1 (2).jpg',
 'cluster.1.jpg',
 'cluster.10 (2).jpg',
 'cluster.10.jpg',
 'cluster.104 (2).jpg',
 'cluster.104.jpg',
 'cluster.106 (2).jpg',
 'cluster.106.jpg',
 'cluster.107 (2).jpg',
 'cluster.107.jpg',
 'cluster.108 (2).jpg',
 'cluster.108.jpg',
 'cluster.109 (2).jpg',
 'cluster.109.jpg',
 'cluster.11 (2).jpg',
 'cluster.11.jpg',
 'cluster.113 (2).jpg',
 'cluster.113.jpg',
 'cluster.12 (2).jpg',
 'cluster.12.jpg',
 'cluster.121 (2).jpg',
 'cluster.121.jpg',
 'cluster.122 (2).jpg',
 'cluster.122.jpg',
 'cluster.124 (2).jpg',
 'cluster.124.jpg',
 'cluster.126 (2).jpg',
 'cluster.126.jpg',
 'cluster.128 (2).jpg',
 'cluster.128.jpg',
 'cluster.13 (2).jpg',
 'cluster.13.jpg',
 'cluster.133 (2).jpg',
 'cluster.133.jpg',
 'cluster.136 (2).jpg',
 'cluster.136.jpg',
 'cluster.137 (2).jpg',
 'cluster.137.jpg',
 'cluster.138 (2).jpg',
 'cluster.138.jpg',
 'cluster.14 (2).jpg',
 'cluster.14.jpg',
 'cluster.142 (2).jpg',
 'cluster.142.jpg',
 'cluster.145 (2).jpg',
 'cluster.145.jpg',
 '

In [70]:
len(dirpath)

37

In [72]:
len(filename)

18

In [ ]:
#!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113

In [87]:
class ImagesDataProcess(Dataset):
    def __init__(self,csv_file,root_dir,tranform=None):
        self.annotation=pd.read_csv(csv_file)
        self.root_dir=root_dir
        self.tranform=tranform
        
    def __len__(self):
        return len(self.annotation)
    
    def __getitem__(self,index):
        img_path=os.path.join(self.root_dir,self.annotation.iloc[index,0])
        image=io.imread(img_path)
        y_label=torch.tensor(int(self.annotation.iloc[index,1]))
        
        if self.tranform:
            image=self.tranform(image)
        return (image,y_label)

In [81]:
my_transform=transforms.Compose(
    [
        #transforms.ToPILImage(),
        #transforms.Resize((224,224)),
#         transforms.ColorJitter(brightness=0,contrast=0,saturation=0,hue=0),
        #transforms.RandomRotation(degrees=45),
        #transforms.RandomHorizontalFlip(p=0.3),
        #transforms.RandomGrayscale(p=0.1),
        #transforms.RandomCrop((224,224)),
        transforms.ToTensor(),
        #transforms.Normalize(mean=[0.5,0.5,0.5],std=[1.0,1.0,1.0])
    ]
) 

In [85]:
from torchvision.transforms import ToTensor
# Hyperparameters
in_channel = 3
num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10
root_dir_path='E:\\JK\\Vt\\dataset_vit_orginal - Copy\\train'
csv_file_path='E:\\JK\\Vt\\file2.csv'

In [88]:
dataset=ImagesDataProcess(
    csv_file=csv_file_path,
    root_dir=root_dir_path,
    tranform=transforms.ToTensor()
)

In [89]:
train_set, test_set = torch.utils.data.random_split(dataset, [1444,350])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

In [90]:
def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(16, 16))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images[:100], 10).permute(1, 2, 0))
        break

In [ ]:
show_batch(train_loader)

In [92]:
class Conv_block(nn.Module):
    def __init__(self,in_channel,out_channel,**kwargs):
        super(Conv_block,self).__init__()
        
        self.network=nn.Sequential(
            nn.Conv2d(in_channel,out_channel,**kwargs),
            nn.BatchNorm2d(out_channel),
            nn.ReLU()
        )
    def forward(self,x):
        return self.network(x)

In [93]:
class Inception_block(nn.Module):
    def __init__(
        self,in_channel,out_1x1,red_3x3,out_3x3,red_5x5,out_5x5,out_pool_3x3):
        
        super(Inception_block,self).__init__()
        
        self.branch_1=nn.Sequential(Conv_block(in_channel,out_1x1,kernel_size=1))
        
        self.branch_2=nn.Sequential(
            Conv_block(in_channel,red_3x3,kernel_size=1),
            Conv_block(red_3x3,out_3x3,kernel_size=3,padding=1)
        )
        
        self.branch_3=nn.Sequential(
            Conv_block(in_channel,red_5x5,kernel_size=1),
            Conv_block(red_5x5,out_5x5,kernel_size=5,padding=2)
        )
        
        self.branch_4=nn.Sequential(
            nn.MaxPool2d(kernel_size=3,stride=1,padding=1),
            Conv_block(in_channel,out_pool_3x3,kernel_size=1)
        )
        
    def forward(self,x):
        return torch.cat(
            [self.branch_1(x),
             self.branch_2(x),
             self.branch_3(x),
             self.branch_4(x)],
            1
        )

In [94]:
class GoogleNet(nn.Module):    
    def __init__(self,in_channel,out_channel):
        super(GoogleNet,self).__init__()
        
        self.conv_1=Conv_block(in_channel,64,kernel_size=7,padding=3,stride=2)
        
        self.pool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        
        self.conv_2=Conv_block(64,192,kernel_size=3,padding=1,stride=1)
        
        self.inception_3a=Inception_block(192,64,96,128,16,32,32)
        self.inception_3b=Inception_block(256,128,128,192,32,96,64)
        
        self.inception_4a=Inception_block(480,192,96,208,16,48,64)
        self.inception_4b=Inception_block(512,160,112,224,24,64,64)
        self.inception_4c=Inception_block(512,128,128,256,24,64,64)
        self.inception_4d=Inception_block(512,112,144,288,32,64,64)
        self.inception_4e=Inception_block(528,256,160,320,32,128,128)
        
        self.inception_5a=Inception_block(832,256,160,320,32,128,128)
        self.inception_5b=Inception_block(832,384,192,384,48,128,128)
        
        self.avgPool=nn.AvgPool2d(kernel_size=7,stride=1)
        
        self.drop=nn.Dropout(p=0.4)
        
        self.fc=nn.Linear(1024,out_channel)
        
        
    def forward(self,x):
        x=self.conv_1(x)
        
        x=self.pool(x)
        
        x=self.conv_2(x)
        
        x=self.pool(x)
        
        x=self.inception_3a(x)
        x=self.inception_3b(x)
        
        x=self.pool(x)
        
        x=self.inception_4a(x)
        x=self.inception_4b(x)
        x=self.inception_4c(x)
        x=self.inception_4d(x)
        x=self.inception_4e(x)
        
        x=self.pool(x)
        
        x=self.inception_5a(x)
        x=self.inception_5b(x)
        
        
        
        x=self.avgPool(x)
        
        x=x.reshape(x.shape[0],-1)

        
        x=self.drop(x)
        
        x=self.fc(x)
        
        return x
        

In [95]:
model=GoogleNet(in_channel=3,out_channel=10)


In [33]:
#from torchsummary import summary
#x=torch.rand(3,224,224)
#print(summary(model,x))

In [96]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [97]:
device=get_default_device()
device

device(type='cpu')

In [98]:
train_loader = DeviceDataLoader(train_loader, device)
test_loader = DeviceDataLoader(test_loader, device)
to_device(model, device);

In [99]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [100]:
train_loader

In [102]:
for epoch in range(num_epochs):
    losses = []

    for data in tqdm(enumerate(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device,dtype=torch.float)
        targets = targets.to(device=device)
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")
%time


0it [00:00, ?it/s]0it [00:12, ?it/s]


AttributeError: 'tuple' object has no attribute 'to'

In [20]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x
            y = y
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()
torch.save(model,'E:\\JK\\Vt\\model_full_trial1.pth')

In [21]:
print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

Checking accuracy on Training Set


100%|██████████| 625/625 [02:41<00:00,  3.86it/s]


Got 18657 / 20000 with accuracy 93.28
Checking accuracy on Test Set


100%|██████████| 157/157 [01:21<00:00,  1.92it/s]

Got 4626 / 5000 with accuracy 92.52


In [1]:
import cv2
import os

def resize_images(folder_path, new_width, new_height):
  """Resizes images in a folder and its subfolders to a new resolution.

  Args:
    folder_path: Path to the folder containing the images.
    new_width: The desired width of the resized images.
    new_height: The desired height of the resized images.
  """
  for dirpath, dirnames, filenames in os.walk(folder_path):
    for filename in filenames:
      if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(dirpath, filename)
        image = cv2.imread(image_path)
        resized_image = cv2.resize(image, (new_width, new_height))
        cv2.imwrite(image_path, resized_image)

# Set the folder path and desired resolution
folder_path = "E:\\JK\\Vt\\dataset_vit_1024"
new_width = 1042
new_height = 1042

# Resize images
resize_images(folder_path, new_width, new_height)

print("Images resized successfully!")
print("1")

Images resized successfully!
1
